In [ ]:
import json

import pandas as pd
from datetime import datetime, timedelta

import trane
import featuretools as ft
import numpy as np

print(ft.__version__)

In [ ]:
df = pd.read_csv("flight-delays/flight-sampled.csv", dtype={"TAIL_NUMBER": str})
df["DATE"] = df["DATE"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df = df.sort_values(by=["DATE"])
df.tail()

In [3]:
# entity_col = "__fake_root_entity__"
# df, meta = trane.overall_prediction_helper(df, meta)
entity_col = "AIRLINE"

# MAP str to int
df_ft = df.copy()
str_col_list = [
    "AIRLINE",
    "FLIGHT_NUMBER",
    "TAIL_NUMBER",
    "ORIGIN_AIRPORT",
    "DESTINATION_AIRPORT",
    "CANCELLATION_REASON",
]
str_mappers = {}
for str_col in str_col_list:
    str_to_id = {}
    id_to_str = []
    n_entity = 0

    for item in set(df_ft[str_col]):
        str_to_id[item] = n_entity
        id_to_str.append(item)
        n_entity += 1

    if str_col == entity_col:
        df[str_col] = df[str_col].apply(lambda x: str_to_id[x])
    df_ft[str_col] = df_ft[str_col].apply(lambda x: str_to_id[x])
    str_mappers[str_col] = (str_to_id, id_to_str)


meta = trane.TableMeta(json.loads(open("flight-delays/meta.json").read()))

df_ft.tail()

,DATE,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE_HOUR,SCHEDULED_TIME,ELAPSED_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
461945,2015-01-31,6,1,6737,443,17,169,10,74.0,88.0,-1.0,13.0,0,3,0.0,0.0,0.0,0.0,0.0
461946,2015-01-31,6,5,14287,2278,161,159,10,256.0,236.0,21.0,1.0,0,3,0.0,0.0,0.0,0.0,0.0
461947,2015-01-31,6,13,8554,3462,104,308,10,148.0,162.0,-9.0,5.0,0,3,0.0,0.0,0.0,0.0,0.0
461938,2015-01-31,6,1,13472,412,150,74,10,85.0,80.0,-6.0,-11.0,0,3,0.0,0.0,0.0,0.0,0.0
469967,2015-01-31,6,5,13335,2002,83,169,23,309.0,312.0,-1.0,2.0,0,3,0.0,0.0,0.0,0.0,0.0


In [4]:
cutoff_base = datetime.strptime("2015-01-06", "%Y-%m-%d")
cutoff_end = datetime.strptime("2015-01-31", "%Y-%m-%d")
cutoff_strategy = trane.FixWindowCutoffStrategy(entity_col, cutoff_base, cutoff_end, 1)

features = trane.FeaturetoolsWrapper(
    df_ft,
    entity_col,
    "DATE",
    {
        "DAY_OF_WEEK": ft.variable_types.Categorical,
        "AIRLINE": ft.variable_types.Categorical,
        "FLIGHT_NUMBER": ft.variable_types.Categorical,
        "TAIL_NUMBER": ft.variable_types.Categorical,
        "ORIGIN_AIRPORT": ft.variable_types.Categorical,
        "DESTINATION_AIRPORT": ft.variable_types.Categorical,
        "CANCELLED": ft.variable_types.Categorical,
        "CANCELLATION_REASON": ft.variable_types.Categorical,
    },
    "flights",
)
features.compute_features(df_ft, cutoff_strategy, 5)

Built 83 features
Elapsed: 00:09 | Progress: 100%|██████████


/Users/sarapido/anaconda3/envs/Trane3.8/lib/python3.8/site-packages/featuretools/entityset/entityset.py:1019: UserWarning: Using training_window but last_time_index is not set on entity flights
  warnings.warn(
/Users/sarapido/anaconda3/envs/Trane3.8/lib/python3.8/site-packages/featuretools/entityset/entityset.py:1019: UserWarning: Using training_window but last_time_index is not set on entity flights
  warnings.warn(
/Users/sarapido/anaconda3/envs/Trane3.8/lib/python3.8/site-packages/featuretools/entityset/entityset.py:1019: UserWarning: Using training_window but last_time_index is not set on entity flights
  warnings.warn(
/Users/sarapido/anaconda3/envs/Trane3.8/lib/python3.8/site-packages/featuretools/entityset/entityset.py:1019: UserWarning: Using training_window but last_time_index is not set on entity flights
  warnings.warn(
/Users/sarapido/anaconda3/envs/Trane3.8/lib/python3.8/site-packages/featuretools/entityset/entityset.py:1019: UserWarning: Using training_window but last_ti

In [5]:
features.features

COUNT(flights)  MAX(flights.AIRLINE_DELAY)  \
AIRLINE time                                                     
0       2015-01-05            7537                      1380.0   
        2015-01-06            7464                      1380.0   
        2015-01-07            7339                      1380.0   
        2015-01-08            7333                      1222.0   
        2015-01-09            7247                      1033.0   
...                            ...                         ...   
13      2015-01-25            3413                       245.0   
        2015-01-26            3450                       251.0   
        2015-01-27            3407                       251.0   
        2015-01-28            3367                       251.0   
        2015-01-29            3452                       257.0   

                    MAX(flights.AIR_SYSTEM_DELAY)  MAX(flights.ARRIVAL_DELAY)  \
AIRLINE time                                                                    
0       2015-01-05                          207.0                      1384.0   
        2015-01-06                          207.0                      1384.0   
        2015-01-07                          207.0                      1384.0   
        2015-01-08                          205.0                      1237.0   
        2015-01-09                          172.0                      1187.0   
...                                           ...                         ...   
13      2015-01-25                          121.0                       249.0   
        2015-01-26                          152.0                       345.0   
        2015-01-27                          152.0                       345.0   
        2015-01-28                          152.0                       345.0   
        2015-01-29                          152.0                       345.0   

                    MAX(flights.DEPARTURE_DELAY)  MAX(flights.ELAPSED_TIME)  \
AIRLINE time                                                                  
0       2015-01-05                        1380.0                      544.0   
        2015-01-06                        1380.0                      544.0   
        2015-01-07                        1380.0                      525.0   
        2015-01-08                        1255.0                      523.0   
        2015-01-09                        1190.0                      520.0   
...                                          ...                        ...   
13      2015-01-25                         245.0                      441.0   
        2015-01-26                         306.0                      441.0   
        2015-01-27                         306.0                      441.0   
        2015-01-28                         306.0                      441.0   
        2015-01-29                         306.0                      438.0   

                    MAX(flights.LATE_AIRCRAFT_DELAY)  \
AIRLINE time                                           
0       2015-01-05                             891.0   
        2015-01-06                             891.0   
        2015-01-07                             891.0   
        2015-01-08                             729.0   
        2015-01-09                             729.0   
...                                              ...   
13      2015-01-25                             146.0   
        2015-01-26                             156.0   
        2015-01-27                             156.0   
        2015-01-28                             156.0   
        2015-01-29                             156.0   

                    MAX(flights.SCHEDULED_DEPARTURE_HOUR)  \
AIRLINE time                                                
0       2015-01-05                                     23   
        2015-01-06                                     23   
        2015-01-07                                     23   
        2015-01-08                           

In [6]:
problem_generator = trane.PredictionProblemGenerator(
    table_meta=meta, entity_col=entity_col, time_col="DATE"
)

problems = problem_generator.generate()

Success/Attempt = 1680/6643


In [7]:
new_df = df[df["DATE"] < "2015-01-15"]
evaluator = trane.PredictionProblemEvaluator(
    new_df,
    entity_col=entity_col,
    cutoff_strategy=cutoff_strategy,
    min_train_set=20,
    min_test_set=20,
    previous_k_as_feature=2,
    latest_k_as_test=8,
)

In [8]:
result = trane.multi_process_evaluation(evaluator, problems, features)
# with open("prob_with_acc.json", "w") as f:
#     json.dump(result, f)

100%|██████████| 1680/1680 [18:43<00:00,  1.50it/s]


In [9]:
result

[{'problem_type': 'regression',
  'template_nl': 'For each <AIRLINE> predict the number of records',
  'evaluations': [{'description': 'no threshold',
    'problem': 'For each <AIRLINE> predict the number of records in next 1 days',
    'label_stats': {'overall_mean': 390.2542857142857,
     'overall_std': 728.5766404799575,
     'entity_std': 522.9609449837188},
    'N_train': 210,
    'N_test': 112,
    'R2': {'LinearRegression': 0.0,
     'DecisionTreeRegressor': 1.0,
     'AdaBoost': 0.0}}]},
 {'problem_type': 'regression',
  'template_nl': 'For each <AIRLINE> predict the number of records with <SCHEDULED_DEPARTURE_HOUR> greater than __',
  'evaluations': [{'description': 'threshold: 16 (keep 25.0%)',
    'problem': 'For each <AIRLINE> predict the number of records with <SCHEDULED_DEPARTURE_HOUR> greater than 16 in next 1 days',
    'label_stats': {'overall_mean': 105.86,
     'overall_std': 197.88314112843756,
     'entity_std': 143.17532655099714},
    'N_train': 210,
    'N_test

In [10]:
import copy

p = copy.deepcopy(problems)
for i in problems:
    # print(i)
    # print(i.operations)
    l = []
    for e in i.operations:
        l.append(e.column_name)
    if len(set(l)) == 1 and None not in l:
        p.remove(i)